In [ ]:
from flask import Flask, render_template, request, jsonify
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import os
import matplotlib.pyplot as plt
import logging
import json
from utils import create_visualizations, load_climate_data

In [5]:
import pandas as pd
import numpy as np
import logging
from sklearn.ensemble import RandomForestClassifier
from flask import Flask, request, render_template

# Initialize Flask app
app = Flask(__name__)
import json

# Configure logging
logging.basicConfig(level=logging.INFO)
df = pd.read_csv('urban_climate_risk_2030.csv')
# Create visualizations
try:
    if df is not None:
        create_visualizations(df)
        logging.info("Visualizations created successfully")
except Exception as e:
    logging.error(f"Error creating visualizations: {str(e)}")

# --- 1. Machine learning model ---
# Use the loaded climate data CSV file for model training
if df is not None:
    try:
        # Convert categorical features
        df['City_Zone_Encoded'] = pd.Categorical(df['City_Zone']).codes

        # Map risk labels to numerical values
        risk_mapping = {'Low': 0, 'Medium': 1, 'High': 2}
        df['risk_level'] = df['Risk_Label'].map(risk_mapping)

        # Select features for model training
        X = df[['Avg_Temperature_C', 'Rainfall_mm', 'Humidity_%', 'Air_Quality_Index',
                'Flood_Risk_Score', 'Heatwave_Risk_Score', 'Drought_Risk_Score', 'City_Zone_Encoded']]
        y = df['risk_level']

        # Train the model
        model = RandomForestClassifier(n_estimators=100, random_state=42)
        model.fit(X, y)

        logging.info(f"Model trained successfully using {len(df)} data points from the CSV file")
    except Exception as e:
        logging.error(f"Error during model training: {str(e)}")
        model = None
else:
    logging.error("Failed to load climate data for model training")
    # Create a simple model with default data as fallback
    model = RandomForestClassifier(n_estimators=10, random_state=42)
    X = np.array([[30, 10, 60, 100, 0.5, 0.5, 0.5, 0]])
    y = np.array([1])  # Medium risk
    model.fit(X, y)

# Get feature importance for visualization
if model is not None:
    try:
        feature_importance = dict(zip(X.columns, model.feature_importances_))
        sorted_feature_importance = {k: v for k, v in sorted(feature_importance.items(), key=lambda item: item[1], reverse=True)}
    except Exception as e:
        logging.error(f"Error calculating feature importance: {str(e)}")
        sorted_feature_importance = {}
else:
    sorted_feature_importance = {}

# Flask route for prediction
@app.route('/predict', methods=['POST'])
def predict():
    try:
        # Get input values from form
        temp = float(request.form['temperature'])
        humidity = float(request.form['humidity'])
        rainfall = float(request.form['precipitation'])
        city_zone = request.form.get('city_zone', 'Zone A')  # Default to Zone A if not provided
        air_quality = float(request.form['air_quality'])
        flood_risk = float(request.form.get('flood_risk', 0.5))  # Default to 0.5 if not provided
        heatwave_risk = float(request.form.get('heatwave_risk', 0.5))  # Default to 0.5 if not provided
        drought_risk = float(request.form.get('drought_risk', 0.5))  # Default to 0.5 if not provided

        # Convert city zone to encoded value
        city_zones = ['Zone A', 'Zone B', 'Zone C', 'Zone D']
        if city_zone not in city_zones:
            city_zone = 'Zone A'  # Default if invalid
        city_zone_encoded = city_zones.index(city_zone)

        # Create feature array for prediction
        features = np.array([[temp, rainfall, humidity, air_quality,
                              flood_risk, heatwave_risk, drought_risk, city_zone_encoded]])

        # Make prediction
        prediction = model.predict(features)[0]
        prediction_proba = model.predict_proba(features)[0]

        # Map prediction to risk level
        risk_labels = ['Low Risk', 'Medium Risk', 'High Risk']
        result = risk_labels[prediction]

        # Generate recommendations based on input values
        recommendations = []
        if prediction > 0:  # Medium or High risk
            if temp > 33:
                recommendations.append("Implement cooling strategies such as green roofs and increased vegetation")
            if humidity < 40:
                recommendations.append("Develop water conservation policies and drought management plans")
            if air_quality > 150:
                recommendations.append("Reduce emissions through improved public transportation and clean energy")
            if flood_risk > 0.7:
                recommendations.append("Enhance flood defenses and improve drainage systems")
            if heatwave_risk > 0.7:
                recommendations.append("Create more shaded areas and cooling centers for vulnerable populations")
            if drought_risk > 0.7:
                recommendations.append("Implement water recycling and rainwater harvesting systems")

        # If no specific recommendations were added, add generic ones
        if not recommendations:
            if prediction == 1:  # Medium risk
                recommendations.append("Monitor climate conditions regularly and develop preparedness plans")
                recommendations.append("Increase urban green spaces to mitigate temperature increases")
            elif prediction == 2:  # High risk
                recommendations.append("Urgent action needed: Develop comprehensive climate adaptation strategy")
                recommendations.append("Invest in climate-resilient infrastructure to protect vulnerable areas")

        # Create prediction results for template
        prediction_results = {
            'risk_level': result,
            'probability': {
                'Low Risk': round(prediction_proba[0] * 100, 2),
                'Medium Risk': round(prediction_proba[1] * 100, 2),
                'High Risk': round(prediction_proba[2] * 100, 2)
            },
            'recommendations': recommendations,
            'input_values': {
                'Temperature': f"{temp}°C",
                'Humidity': f"{humidity}%",
                'Rainfall': f"{rainfall}mm",
                'Air Quality Index': air_quality,
                'City Zone': city_zone
            }
        }

        return render_template('index.html',
                               prediction_text=f"Predicted Climate Risk: {result}",
                               prediction_results=prediction_results,
                               feature_importance=json.dumps(sorted_feature_importance))
    except Exception as e:
        logging.error(f"Prediction error: {str(e)}")
        return render_template('index.html',
                               prediction_text=f"Error: {str(e)}",
                               feature_importance=json.dumps(sorted_feature_importance))


ERROR:root:Error creating visualizations: name 'create_visualizations' is not defined
INFO:root:Model trained successfully using 53 data points from the CSV file
